<a href="https://colab.research.google.com/github/sprksh-crypto/MachineLearningProjects/blob/transfer/buildAndTrainBirdsVsSquirrels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This code is to differentiate between 1. Birds (no squirrels/chipmunks) 2. Squirrel/Chipmunk
#3. feeder or tree without squirrels, chipmunks, or birds.
#Mount drive.

from google.colab import drive

#for authorisation
drive.mount('/content/drive')

#Drive files in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

import tensorflow as tf
import numpy as np

Mounted at /content/drive
'1 Research 5_3_23.ipynb'		  'ML Class Code 1 (Flowers).ipynb'
 birderModel				  'ML Lab 1'
 birds-10-eachOf-358.tfrecords		  'ML textbook code 1.ipynb'
 birds-20-eachOf-358.tfrecords		  'Physics Project'
 birds-vs-squirrels-train.tfrecords	  'Physics Scripts'
 birds-vs-squirrels-validation.tfrecords   Pics
 buildtrainBirdsvSquirrels.ipynb	   Untitled1.ipynb
'Essay Drafts'


In [ ]:
filepath=['/content/drive/My Drive/birds-vs-squirrels-train.tfrecords']
filepath1=['/content/drive/My Drive/birds-vs-squirrels-validation.tfrecords']

raw_birdSquirrel_training_dataset=tf.data.TFRecordDataset(filepath)
raw_birdSquirrel_validation_dataset=tf.data.TFRecordDataset(filepath1)

In [ ]:
#Need to create testing set from given sets.
datLen=raw_birdSquirrel_training_dataset.reduce(0,lambda x,y: x+1)

n_train=int(datLen.numpy()*.85)
n_test=int(datLen.numpy()*.15)

In [ ]:
feature_description={'image':tf.io.FixedLenFeature([],tf.string),'label':tf.io.FixedLenFeature([],tf.int64)}

#Define parse example to load data.

def parse_examples(serialized_examples):
  examples=tf.io.parse_example(serialized_examples,feature_description)
  targets=examples.pop('label')
  images=tf.image.resize_with_pad(tf.cast(tf.io.decode_jpeg(examples['image'],channels=3),tf.float32),299,299)
  return images, targets

In [ ]:
#Load in dataset.
#But do I want to load it as supervised.
training_dataset_BS=raw_birdSquirrel_training_dataset.take(n_train).map(parse_examples,num_parallel_calls=tf.data.AUTOTUNE)
validation_dataset_BS=raw_birdSquirrel_validation_dataset.map(parse_examples,num_parallel_calls=tf.data.AUTOTUNE)

testing_dataset_BS=raw_birdSquirrel_training_dataset.skip(n_train).take(n_test).map(parse_examples,num_parallel_calls=tf.data.AUTOTUNE)

In [ ]:
#Print to see if working.
#for el in training_dataset_BS.take(1):
#  print(el)

In [ ]:
#Preprocessing layer.

def preproc(image,label):
  inp=tf.keras.applications.xception.preprocess_input(image)
  return inp,label

In [ ]:
#Apply preprocessing layer.

training_dataset_BS1=training_dataset_BS.take(n_train).map(preproc,num_parallel_calls=tf.data.AUTOTUNE).batch(64)
validation_dataset_BS1=validation_dataset_BS.map(preproc,num_parallel_calls=tf.data.AUTOTUNE).batch(64)

testing_dataset_BS1=testing_dataset_BS.skip(n_train).take(n_test).map(preproc,num_parallel_calls=tf.data.AUTOTUNE).batch(64)

In [ ]:
#Defining model.

#Remove top layer and look.

base_model=tf.keras.applications.xception.Xception(weights='imagenet',include_top=False)
#base_model.summary()

In [ ]:
#Add new top layer.
avg=tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output=tf.keras.layers.Dense(3,activation="softmax")(avg)
model=tf.keras.models.Model(inputs=base_model.input,outputs=output)
#model.summary()

In [ ]:
#Freeze the weights of the lower layers.
#Why when print, the last two are true? SHouldn't it just be the last?

for layer in base_model.layers:
  layer.trainable=False

#for layer in model.layers:
#  print(layer.trainable)

In [ ]:
#Fit the new upper layer.

checkpoint_cb=tf.keras.callbacks.ModelCheckpoint('BirdSquirrelMod-topFit.tgz',save_best_only=True)

earlyStop_cb=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

ss=0.001

opt=tf.keras.optimizers.SGD(learning_rate=ss)

model.compile(loss="sparse_categorical_crossentropy",optimizer=opt,metrics=["accuracy"])

model.fit(training_dataset_BS1,validation_data=validation_dataset_BS1,epochs=20,callbacks=[checkpoint_cb,earlyStop_cb])

Epoch 1/20
    109/Unknown - 52s 455ms/step - loss: 0.3556 - accuracy: 0.9482

109/109 [==============================] - 88s 788ms/step - loss: 0.3556 - accuracy: 0.9482 - val_loss: 0.3593 - val_accuracy: 0.9582
Epoch 2/20
109/109 [==============================] - ETA: 0s - loss: 0.3368 - accuracy: 0.9502

109/109 [==============================] - 82s 757ms/step - loss: 0.3368 - accuracy: 0.9502 - val_loss: 0.3411 - val_accuracy: 0.9597
Epoch 3/20
109/109 [==============================] - ETA: 0s - loss: 0.3205 - accuracy: 0.9524

109/109 [==============================] - 82s 755ms/step - loss: 0.3205 - accuracy: 0.9524 - val_loss: 0.3252 - val_accuracy: 0.9601
Epoch 4/20
109/109 [==============================] - ETA: 0s - loss: 0.3061 - accuracy: 0.9531

109/109 [==============================] - 85s 781ms/step - loss: 0.3061 - accuracy: 0.9531 - val_loss: 0.3112 - val_accuracy: 0.9609
Epoch 5/20
109/109 [==============================] - ETA: 0s - loss: 0.2934 - accuracy: 0.9540

...

109/109 [==============================] - 87s 796ms/step - loss: 0.2118 - accuracy: 0.9623 - val_loss: 0.2187 - val_accuracy: 0.9653
Epoch 17/20
109/109 [==============================] - ETA: 0s - loss: 0.2073 - accuracy: 0.9629

109/109 [==============================] - 86s 788ms/step - loss: 0.2073 - accuracy: 0.9629 - val_loss: 0.2143 - val_accuracy: 0.9649
Epoch 18/20
109/109 [==============================] - ETA: 0s - loss: 0.2031 - accuracy: 0.9632

109/109 [==============================] - 79s 729ms/step - loss: 0.2031 - accuracy: 0.9632 - val_loss: 0.2102 - val_accuracy: 0.9653
Epoch 19/20
109/109 [==============================] - ETA: 0s - loss: 0.1992 - accuracy: 0.9636

109/109 [==============================] - 83s 762ms/step - loss: 0.1992 - accuracy: 0.9636 - val_loss: 0.2064 - val_accuracy: 0.9661
Epoch 20/20
109/109 [==============================] - ETA: 0s - loss: 0.1954 - accuracy: 0.9644

109/109 [==============================] - 82s 756ms/step - loss: 0.1954 - accuracy: 0.9644 - val_loss: 0.2027 - val_accuracy: 0.9661


In [ ]:
tf.saved_model.save(model, '/content/drive/My Drive/birdsVsSquirrelsModel')